# Late contributions Received and Made

## Setup

In [2]:
%load_ext sql

In [3]:
from django.conf import settings
connection_string = 'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(
    **settings.DATABASES['default']
)
%sql $connection_string

u'Connected: ccdc@calaccess_website'

## Unique Composite Key

The documentation says that the records are unique on the following fields:
* `FILING_ID`
* `AMEND_ID`
* `LINE_ITEM`
* `REC_TYPE`
* `FORM_TYPE`

`REC_TYPE` is always the same value: `S497`, so we can ignore this column. 

`FORM_TYPE` is either `F497P1` or `F497P2`, indicating in whether itemized transaction is listed under Part 1 (Contributions Received) or Part 2 (Contributions Made). I'll split these up into separate tables.

## Are the `S497_CD` records actually unique on `FILING_ID`, `AMEND_ID` and `LINE_ITEM`?

Yes. And this is even true across the Parts 1 and 2 (Contributions Received and Contributions Made).

In [11]:
%%sql
SELECT "FILING_ID", "AMEND_ID", "LINE_ITEM", COUNT(*)
FROM "S497_CD"
GROUP BY 1, 2, 3
HAVING COUNT(*) > 1
ORDER BY COUNT(*) DESC;

0 rows affected.


FILING_ID,AMEND_ID,LINE_ITEM,count


## `TRAN_ID`

The `S497_CD` table includes a `TRAN_ID` field, which the [documentation](http://calaccess.californiacivicdata.org/documentation/calaccess-files/s497-cd/#fields) describes as a "Permanent value unique to this item".

### Is `TRAN_ID` ever `NULL` or blank?

No.

In [10]:
%%sql
SELECT COUNT(*)
FROM "S497_CD"
WHERE "TRAN_ID" IS NULL OR "TRAN_ID" = '' OR "TRAN_ID" = '0';

1 rows affected.


count
0


### Is `TRAN_ID` unique across filings?

Decidedly no.

In [3]:
%%sql
SELECT "TRAN_ID", COUNT(DISTINCT "FILING_ID") 
FROM "S497_CD"
GROUP BY 1
HAVING COUNT(DISTINCT "FILING_ID") > 1
ORDER BY COUNT(DISTINCT "FILING_ID") DESC
LIMIT 100;

100 rows affected.


TRAN_ID,count
1,14271
2,5893
3,3490
4,2336
5,1661
6,1139
7,848
8,659
01,627
9,534


But `TRAN_ID` does appear to be unique within each filing amendment, and appears to be reused for each filing.

In [7]:
%%sql
SELECT "FILING_ID", "TRAN_ID", COUNT(DISTINCT "AMEND_ID") AS amend_count, COUNT(*) AS row_count
FROM "S497_CD"
GROUP BY 1, 2
ORDER BY COUNT(*) DESC
LIMIT 100;

100 rows affected.


FILING_ID,TRAN_ID,amend_count,row_count
1896576,C10522528,6,6
1361544,NON98,6,6
1361544,NON92,6,6
1896576,C10522532,6,6
1058082,C1742A,6,6
1361544,NON100,6,6
1058082,C1743,6,6
1361544,NON95,6,6
1361544,NON99,6,6
977789,F497P10000244485,6,6


There's one exception:

In [8]:
%%sql
SELECT "FILING_ID", "TRAN_ID", "AMEND_ID", COUNT(*)
FROM "S497_CD"
GROUP BY 1, 2, 3
HAVING COUNT(*) > 1;

1 rows affected.


FILING_ID,TRAN_ID,AMEND_ID,count
2072379,EXP9671,1,2


Looks like this `TRAN_ID` is duplicated across the two parts of the filing. So it was both a contribution both made and received?

In [5]:
%%sql
SELECT *
FROM "S497_CD"
WHERE "FILING_ID" = 2072379
AND "TRAN_ID" = 'EXP9671';

2 rows affected.


id,FILING_ID,AMEND_ID,LINE_ITEM,REC_TYPE,FORM_TYPE,TRAN_ID,ENTITY_CD,ENTY_NAML,ENTY_NAMF,ENTY_NAMT,ENTY_NAMS,ENTY_CITY,ENTY_ST,ENTY_ZIP4,CTRIB_EMP,CTRIB_OCC,CTRIB_SELF,ELEC_DATE,CTRIB_DATE,DATE_THRU,AMOUNT,CMTE_ID,CAND_NAML,CAND_NAMF,CAND_NAMT,CAND_NAMS,OFFICE_CD,OFFIC_DSCR,JURIS_CD,JURIS_DSCR,DIST_NO,OFF_S_H_CD,BAL_NAME,BAL_NUM,BAL_JURIS,MEMO_CODE,MEMO_REFNO,BAL_ID,CAND_ID,SUP_OFF_CD,SUP_OPP_CD
62013206,2072379,1,2,S497,F497P1,EXP9671,COM,McCarty for Assembly 2016,,,,Sacramento,CA,95814,,,,None,2016-08-25,None,1000.00,1373943,,,,,,,,,,,,,,,EXP:A:9671,,,,
62013205,2072379,1,1,S497,F497P2,EXP9671,COM,McCarty for Assembly 2016,,,,Sacramento,CA,95814,,,,2016-11-08,2016-08-25,None,-1000.00,1373943,Assembly Member Kevin McCarty,,,,ASM,,ASM,,07,H,,,,,EXP:S497:9671,,,,


Looking at the [PDF for the filing](http://cal-access.ss.ca.gov/PDFGen/pdfgen.prg?filingid=2072379&amendid=1), it appears to be a check from the California Psychological Association PAC to the McCarty for Assembly 2016 committee, which was given and returned on 8/25/2016.

Regardless, because the combinations of `FILING_ID`, `AMEND_ID` and `TRAN_ID` are unique within each part of the Schedule 497, we could substitute `TRAN_ID` for `LINE_ITEM` in the composite key when splitting up the contributions received from the contributions made.

The advantage is that the `TRAN_ID` purportedly points to the same contribution from one amendment to the next, whereas the same `LINE_ITEM` might not because the filers don't necessarily list transactions on the same line from one filing amendment to the next.

Here's an example: On the [original Schedule 497 filing](http://cal-access.ss.ca.gov/PDFGen/pdfgen.prg?filingid=2083478&amendid=0) for Steven Bradford for Senate 2016, a $8,500.00 contribution from an AFL-CIO sub-committee is listed on line 1. But on the [first](http://cal-access.ss.ca.gov/PDFGen/pdfgen.prg?filingid=2083478&amendid=1) and [second](http://cal-access.ss.ca.gov/PDFGen/pdfgen.prg?filingid=2083478&amendid=2) amendments to the filing, it is listed on line 4.
